In [1]:
'''
stock_ids = ['asianpaint','axisbank','bergepaint','bhartiartl','canbk','dmart','federalbnk','havells','hdfcbank',
             'hdfclife','hindunilvr','ibrealest','icicibank','indusindbk','infy','itc','mhril','motilalofs',
             'nationalum','ntpc','reliance','sail','tataconsum','tatamotors','tatapower','tatasteel','tcs','zeel']

# 'alkylamine','irctc', 'paras'
'''

"\nstock_ids = ['asianpaint','axisbank','bergepaint','bhartiartl','canbk','dmart','federalbnk','havells','hdfcbank',\n             'hdfclife','hindunilvr','ibrealest','icicibank','indusindbk','infy','itc','mhril','motilalofs',\n             'nationalum','ntpc','reliance','sail','tataconsum','tatamotors','tatapower','tatasteel','tcs','zeel']\n\n# 'alkylamine','irctc', 'paras'\n"

In [2]:
import pandas as pd

mypf = pd.read_csv('data/myPortfolio.csv')

stock_ids = mypf[(mypf['type'] == 'stock') & (mypf['id'] != 'niftybees')]['id'].unique()

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if 'Dec 2021' in test_list:
        quarter = 'Dec 2021'
    else:
        quarter = 'Sep 2021'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

3iinfoltd
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11

[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
canbk
189
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[**********

Exception in thread Thread-134:
Traceback (most recent call last):
  File "C:\Users\ankit.rathi\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\ankit.rathi\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "C:\Users\ankit.rathi\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "C:\Users\ankit.rathi\Anaconda3\lib\site-packages\urllib3\connection.py", line 411, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\ankit.rathi\Anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 428, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\ankit.rathi\Anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 472, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\ankit.rathi\Anaconda3

KeyboardInterrupt: 

In [ ]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(TTM)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS'])*100/tmp['Prev_EPS'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(TTM)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS'])*100/tmp['Prev_EPS'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

In [ ]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2021-11-15'].sort_values(by='Sales%', ascending=False)

In [ ]:
df_growth[df_growth['Stock'] == 'TIPSINDLTD.NS']

In [ ]:
df_analysis_final.to_csv('data/Portfolio_FvS_QoQ_Analysis.csv', index=False)

In [ ]:
df_growth = pd.read_csv('data/Portfolio_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

In [ ]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

In [ ]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [ ]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

In [ ]:
df_growth[df_growth['Stock'] == 'TATASTEEL.NS'][cols]